In [ ]:
## download project & convert TF1 to TF2.
!git clone https://github.com/JiahuiYu/generative_inpainting
!tf_upgrade_v2 --intree generative_inpainting --outtree generative_inpainting_tf2

## download support module & convert TF1 to TF2.
!git clone https://github.com/JiahuiYu/neuralgym
!tf_upgrade_v2 --intree neuralgym/neuralgym/ --outtree generative_inpainting_tf2/neuralgym

In [ ]:
## apply patch

%%bash
cd /content/generative_inpainting_tf2/
cat << EOS | git apply
diff --git a/inpaint_model.py b/inpaint_model.py
index 6e8ca5d..93ccf2e 100644
--- a/inpaint_model.py
+++ b/inpaint_model.py
@@ -4,7 +4,7 @@ import logging
 import cv2
 import neuralgym as ng
 import tensorflow as tf
-from tensorflow.contrib.framework.python.ops import arg_scope
+from tf_slim import arg_scope
 
 from neuralgym.models import Model
 from neuralgym.ops.summary_ops import scalar_summary, images_summary
diff --git a/inpaint_ops.py b/inpaint_ops.py
index 740ac46..66ef60b 100644
--- a/inpaint_ops.py
+++ b/inpaint_ops.py
@@ -4,7 +4,7 @@ import math
 import cv2
 import numpy as np
 import tensorflow as tf
-from tensorflow.contrib.framework.python.ops import add_arg_scope
+from tf_slim import add_arg_scope
 from PIL import Image, ImageDraw
 
 from neuralgym.ops.layers import resize
diff --git a/neuralgym/ops/layers.py b/neuralgym/ops/layers.py
index d4fabbc..ed35226 100644
--- a/neuralgym/ops/layers.py
+++ b/neuralgym/ops/layers.py
@@ -494,4 +494,4 @@ def flatten(x, name='flatten'):
 
     """
     with tf.compat.v1.variable_scope(name):
-        return tf.contrib.layers.flatten(x)
+        return tf.compat.v1.layers.flatten(x)
EOS

In [ ]:
!gdown -O generative_inpainting_tf2/model --folder https://drive.google.com/drive/folders/1y7Irxm3HSHGvp546hZdAZwuNmhLUVcjO?usp=sharing 

In [ ]:
## test run

%cd /content/generative_inpainting_tf2/
!python test.py --image examples/places2/case1_input.png --mask examples/places2/case1_mask.png --output case1_output.png --checkpoint_dir model

In [ ]:
## convert the model to tflite

import numpy as np
import tensorflow as tf
import neuralgym as ng
from inpaint_model import InpaintCAModel

CHECKPOINT = './model'
TFLITE     = 'deepfillv2.tflite'

FLAGS = ng.Config('inpaint.yml')
model = InpaintCAModel()

def saturate_cast_u8(x):
  x = tf.math.maximum(x, tf.constant([0.0], dtype=tf.float32))
  x = tf.math.minimum(x, tf.constant([255.0], dtype=tf.float32))
  x = tf.cast(x, dtype=tf.uint8)
  return x

sess_config = tf.compat.v1.ConfigProto()
sess_config.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=sess_config) as sess:
  input_image = tf.cast(tf.random.uniform([1,512,1360,3], maxval=255, dtype=tf.int32), tf.float32)
  output = model.build_server_graph(FLAGS, input_image)
  output = (output + 1.) * 127.5
  output = tf.reverse(output, [-1])
  output = saturate_cast_u8(output)
  # load pretrained model
  vars_list = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES)
  assign_ops = []
  for var in vars_list:
    vname = var.name
    var_value = tf.train.load_variable(CHECKPOINT, vname)
    assign_ops.append(tf.compat.v1.assign(var, var_value))
  sess.run(assign_ops)
  print('Model loaded.')

  # save the model as saved_model
  tf.compat.v1.saved_model.simple_save(
    sess,
    './saved_model',
    inputs={'input': input_image},
    outputs={'output': output})

# convert to tflite
converter = tf.lite.TFLiteConverter.from_saved_model('./saved_model')
converter.allow_custom_ops = True
tflite_model = converter.convert()

open(TFLITE, "wb").write(tflite_model)

In [ ]:
## download the converted model - include saved_model & tflite

!zip -r tfl_deepfillv2.zip saved_model deepfillv2.tflite
from google.colab import files
files.download('/content/generative_inpainting_tf2/tfl_deepfillv2.zip')

In [ ]:
## download project

!cd ..; zip -r generative_inpainting_tf2.zip ./generative_inpainting_tf2
files.download('/content/generative_inpainting_tf2.zip')